# Patterns for Cleaner Python

## Assertion

assertion is about unrecoverable errors; while exception can happen and needs to be handled. assertion helps debug but not runtime errors. the python interpreter translate 

```python
assert expression1, expression2
```

to

```python
if __debug__:       # global variable, False when optimized
    if not expression1:
        raise AssertionError(expression2)
```


assert is globally disabled with -o, -oo command line swithes, and PYTHONOPTIMIZE enviroment variable in CPython, in which case assert is compiled away. So be sure not use assert for prod validation of data.

In [4]:
# never pass a tuple as first argument in assert, it will never fail:
assert (1==2, 'this will never fail')

## Complacent Comma Placement

one item per line, use comma.

In [5]:
names = [
    'alice',
    'bob',
    'charlie',
    'jane',
]

## Context Managers and the `with` Statement

`with` helps with resource management. It simplifies explicity 'try... finally' statement.

### supporting `with` in objects

implement `__enter__` and `__exit__` context manager. python calls `__enter__` when execution enters the context of `with`, then call `__exit__` when resource got freed.



In [6]:
class ManageFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self):
        if self.file:
            self.file.close()

another way is to use std library `contextlib.contextmanager` decorator to define a generator-based factory function for resource. It will automatically support with statement.

In [8]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('hw')

In [11]:
class Indenter:
    def __init__(self):
        self.level = 0

    def __enter__(self):
        self.level += 1
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1

    def print(self, text):
        print('    ' * self.level + text)

with Indenter() as indent:
    print(indent)
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('world')
    indent.print('hey') 

None


AttributeError: 'NoneType' object has no attribute 'print'

In [10]:
indent = Indenter()
indent.print('1')

1


## Underscores

### single leading `_`

suggested private 

### double leading `__`

python would change such variable name to a new one. a form of stronger suggested private. check examples

### double leading/trailing `__`

python would not mangle such variables, leave it for magic methods

### single `_`

just a variable name, REPL use it to reference last variable


## String formating

1. old c-style 
2. python style
3. formatted string literal
4. template string (good for user input, for security reasons)

In [19]:
errno = 50159747054
name = 'Bob'

# c-style
print('hello, %s' % name)
print('%x' % errno)
print('hey %s, there is 0x%x error!' % (name, errno))
print('hey %(name)s, there is a 0x%(errno)x error!' % {'name':name, 'errno': errno})

# python-style
print('hello, {}'.format(name))
print('hey {name}, there is a 0x{errno:x} error!'.format(name=name, errno=errno))

# formatted string literals
print(f'Hello, {name}')
a, b = 5, 10
print(f'can do calculations: {a + b} and {2 * (a+b)}')
print(f'hey {name}, there is a {errno:#x} error!')

# template string
from string import Template
t = Template('hey, $name')
t.substitute(name=name)

templ_string = 'hey $name, there is a $error error'
Template(templ_string).substitute(name=name, error=hex(errno))

# template string is safe
PASSWORD = 'pass'
class Error:
    def __init__(self):
        pass
err = Error()
user_input = '{error.__init__.__globals__[PASSWORD]}'
user_input.format(error=err)

user_input = '${error.__init__.__globals__[PASSWORD]}'
try:
    Template(user_input).substitute(error=err)
except ValueError as e:
    print(e)

hello, Bob
badc0ffee
hey Bob, there is 0xbadc0ffee error!
hey Bob, there is a 0xbadc0ffee error!
hello, Bob
hey Bob, there is a 0xbadc0ffee error!
Hello, Bob
can do calculations: 15 and 30
hey Bob, there is a 0xbadc0ffee error!
Invalid placeholder in string: line 1, col 1


In [21]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


# Effective Functions

## python function are first class



In [23]:
def yell(text):
    return text.upper() + '!'

bark = yell

# functions passed to other functions
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

greet(bark)

# function can be nested
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

# but whisper does not exist outside speak
try:
    whisper('Yo')
except NameError as e:
    print(e)

try:
    speak.whisper
except AttributeError as e:
    print(e) 

HI, I AM A PYTHON PROGRAM!
name 'whisper' is not defined
'function' object has no attribute 'whisper'


### functions can capture local state

the inner functions can capture parent function's state. Functions that do this are called lexical closures (or just closures, for short). A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.

you can pre-config some behaviors to the function you return.

In [25]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '...'
    if volume > 0.5:
        return yell
    else:
        return whisper

print(get_speak_func('Hello', 0.7)())

'HELLO...'

### objects can behave like functions through `__call__`

In [27]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x

plus_3 = Adder(3)
print(plus_3(4))
print(callable(plus_3))

7
True


## Lambdas

it is a single-expression function that are not necessarily bound to a name.

it has lexical scope that captures context


In [3]:
(lambda x, y: x + y)(3, 4)

def make_adder(n):
    return lambda x: x + n

plus_3 = make_adder(3)
plus_5 = make_adder(5)

plus_3(4)

7

## Decorators

* It uses the * and ** operators in the wrapper closure definition to collect all positional and keyword arguments and stores them in variables (args and kwargs).

* The wrapper closure then forwards the collected arguments to the original input function using the * and ** “argument unpacking” operators.

* use functional.wrap to keep meta data



In [4]:
# multiple decorator order

def strong(func):
    def wrapper():
        return 'strong' + func()
    return wrapper

def emphasis(func):
    def wrapper():
        return 'emphy' + func()
    return wrapper

@strong
@emphasis
def greet():
    return "Hello"

greet()

'strongemphyHello'

In [ ]:
# pass arguments

def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [5]:
# loss of meta data
def greet():
    """ greetings """

def uppercase(func):
    def wrapper():
        return func().upper()
    return wrapper

decor_greet = uppercase(greet)
decor_greet.__name__

'wrapper'

In [6]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

@uppercase
def greet():
    """ greetter"""
    return 'hello!'

greet.__name__

'greet'

## `*args` and `**kwargs`

forwarding optional and keyword args.

In [ ]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

## Function Argument Unpacking

all iterable, including generaotr expressions can be unpacked with *. all keyword arguments dictionary can be unpacked with **. Note * on kwargs will unpack keys.

In [11]:
def print_3num(x, y, z):
    print ('<%s, %s, %s>' % (x, y, z))

tup_vec = (1, 0, 1)
print_3num(*tup_vec)

gen_vec = (x * x for x in range(3))
print_3num(*gen_vec)

dic_vec = {'x': 1, 'y':3, 'z': 5}
print_3num(**dic_vec) # unpact values
print_3num(*dic_vec) # unpact keys

<1, 0, 1>
<0, 1, 4>
<1, 3, 5>
<x, y, z>


# Classes and OOP
## is vs ==
* `==` checks equality
* `is` checks address

In [2]:
a = [1, 2, 3]
b = a

print(a == b)
print(a is b)

c = list(a)
print(c == a)
print (c is a)

True
True
True
False


## every class needs a `__repr__`

`__str__` is not triggered when stack unwinding. print will call `__repr__` if __str__ is not implemented. 

In [6]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __repr__(self):
        return '__repr__ for Car'

    def __str__(self):
        return '__str__ for Car'

my_car = Car('red', 1234)
print(my_car)
print('{}'.format(my_car))
print(str([my_car]))
my_car

__str__ for Car
__str__ for Car
[__repr__ for Car]


__repr__ for Car

In [9]:
# the !r conversion flag make sure the output string uses repr(self.color) instead of str(self.color)
def __repr__(self):
    return (f'{self.__class__.__name__}('f'{self.color!r}, {self.mileage!r})')

## define own exception

In [10]:
class NameTooshort(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooshort(name)

validate('a')

NameTooshort: a

## cloning objects

build in copy is shallow
```python
newlist = list(originallist)
newdict = dict(originaldict)
newset = set(originalset)
```

use `deepcopy` to deepcopy, use `copy` to explicit shallow copy for customized types, use list/dict/set to shallow copy.

you can control the class behavior using `__copy__()` and `__deepcopy__()`

In [13]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
ys = list(xs)
xs[1][0] = 'x'
print(ys)

import copy
zs = copy.deepcopy(xs)
xs[1][1]='u'
print(zs)

[[1, 2, 3], ['x', 5, 6], [7, 8, 9]]
[[1, 2, 3], ['x', 5, 6], [7, 8, 9]]


In [17]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'

a = Point(23, 42)
b = copy.copy(a)         # the primitive type shallow = deep

print(a is b)
a.x = 32

print(b)

False
Point(23, 42)


In [21]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft
        self.bottomright = bottomright

    def __repr__(self):
        return (f'Rectangle({self.topleft!r}, 'f'{self.bottomright!r})')

r1 = Rectangle(Point(0, 1), Point(5, 6))
r2 = copy.copy(r1)
r3 = copy.deepcopy(r1)

r1.topleft.x = 999
print(r1)
print(r2)
print(r3)


Rectangle(Point(999, 1), Point(5, 6))
Rectangle(Point(999, 1), Point(5, 6))
Rectangle(Point(0, 1), Point(5, 6))


## Abstract Base Classes

we need the to define the interface
1. instantiating the base class is impossible
2. forgetting implementing any interface methods will raise error

In [22]:
from abc import ABCMeta, abstractclassmethod

class Base(metaclass=ABCMeta):
    @abstractclassmethod
    def foo(self):
        pass

    @abstractclassmethod
    def bar(self):
        pass

class Concrete(Base):
    def foo(self):
        pass
    # no bar()

c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

## Namedtuples for read only dictionary

tuple is good, but you can not name index it. 
* 1st string property is the variable name, good for debug __repr__
* space delimited string is a syntax sugar for \[color, mileage\]

In [28]:
from collections import namedtuple

Car = namedtuple('Car', 'color mileage')
my_car = Car('red', 3000)
print(my_car.color)
print(my_car.mileage)
print(my_car[0])
tuple(my_car)

color, mileage = my_car
print(color)
print(mileage)
print(my_car)

red
3000
red
red
3000
Car(color='red', mileage=3000)


In [29]:
# property is const
my_car.color = 'blue'

AttributeError: can't set attribute

the proper way to subclass a named tuple and add more constant attributes are not subclass, but use `_fields`

In [31]:
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge',))
ElectricCar('red', 3222, 45.0)

ElectricCar(color='red', mileage=3222, charge=45.0)

there are many built-in helpers starts with `_`:
* `_asdict` turn into dictionary
* `_replace` shallow copy a tuple and allows to selectively replace some fields
* `_make` turns a iteratable (tuple, list, set) into a namedtuple

In [34]:
print(my_car._asdict())

print(my_car._replace(color='blue'))

print(Car._make(['red', 999]))

OrderedDict([('color', 'red'), ('mileage', 3000)])
Car(color='blue', mileage=3000)
Car(color='red', mileage=999)


## Class vs Instanve Variable Pitfalls

* class variables are shared among instances
* instance can read only class variables, but once it sets value, it will create a instance level varialbe, and shaddow the class variable 

In [40]:
class Dog:
    numlegs = 4
    def __init__(self, name):
        self.name = name

jack = Dog('jack')
jill = Dog('jill')

print(jack.numlegs, jill.numlegs, Dog.numlegs)

Dog.numlegs = 6
print(jack.numlegs, jill.numlegs, Dog.numlegs)

Dog.numlegs = 4
jack.numlegs = 6        # shaddows class property
print(jack.numlegs, jill.numlegs, Dog.numlegs)

print(jack.numlegs, jack.__class__.numlegs)


4 4 4
6 6 6
6 4 4
6 4


## Instance, Class and Static Methods 

* instance methods can access class itself through `self._class__` attribute, so that to modify class state
* class methods only has access to cls, it cannot modify state of instance, but it can modify class state across all instances.
* static method cannot modify object state or class state. it is restricted in what data they can access, they're primarily used as namespace your methods. 

In [43]:
class MyClass:
    def method(self):
        return 'instance method called', self

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'

obj = MyClass()
obj.method()

('instance method called', <__main__.MyClass at 0x7f303d3cab90>)

In [45]:
# alternative way to call obj.method(), explicity pass obj to self
MyClass.method(obj) 

('instance method called', <__main__.MyClass at 0x7f303d3cab90>)

In [47]:
# obj access self.__class__ attribute and call the class method
obj.classmethod()

('class method called', __main__.MyClass)

In [48]:
# obj access static method
obj.staticmethod()

'static method called'

In [50]:
MyClass.classmethod()

('class method called', __main__.MyClass)

In [52]:
MyClass.staticmethod()


'static method called'

In [53]:
MyClass.method()

TypeError: method() missing 1 required positional argument: 'self'

### class method for factory pattern

python allow one `__init__` per class, but class methods can make many constructors

In [9]:
import math

class Pizza:
    def __init__(self, radius, ingredients):
        self.radius = radius
        self.ingredients = ingredients
    def __repr__(self):
        return (f'Pizza({self.radius!r}, {self.ingredients!r})')

    @classmethod
    def margherita(cls):
        return cls(2, ['mozzarella', 'tomatoes'])
    @classmethod
    def prosciutto(cls):
        return cls(4, ['mozzarella', 'tomatoes', 'ham'])

    def area(self):
        return self.circle_area(self.radius)
    
    # not allow to modify class state
    @staticmethod
    def circle_area(r):
        return r ** 2 * math.pi

print(Pizza.margherita())

p = Pizza.prosciutto()
print(p)

print(p.area())
Pizza.circle_area(4)

Pizza(2, ['mozzarella', 'tomatoes'])
Pizza(4, ['mozzarella', 'tomatoes', 'ham'])
50.26548245743669


50.26548245743669